In [ ]:
!pip install xgboost
!pip install -U scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
train_data = pd.read_csv('training_data.csv')
y_train = train_data['Performance Index']
x_train = train_data.drop('Performance Index', axis=1)
x_train = x_train.to_numpy()

In [ ]:
model = XGBRegressor()
model.fit(x_train, y_train)

In [ ]:
def accuracy(test_data_file = 'test_data.csv'):
    test_data = pd.read_csv(test_data_file)
    y_test = test_data['Performance Index']
    x_test = test_data.drop('Performance Index', axis=1)
    x_test = x_test.to_numpy()
    y_pred = model.predict(x_test)
    y_test, y_pred = y_test*100, y_pred*100
    print(f'MAE:{mean_absolute_error(y_test, y_pred)} performance index points')

In [ ]:
def stratified_accuracy(test_data_file = 'test_data.csv',
                        percentiles_performance_index = [0.4 ,0.55,0.71]):
    test_data = pd.read_csv(test_data_file)
    y_test = test_data['Performance Index']
    x_test = test_data.drop('Performance Index', axis=1)
    y_test =  y_test*100
    percentiles_performance_index = np.array(percentiles_performance_index)*100
    x_test = x_test.to_numpy()
    
    high_performance_cond = y_test >= percentiles_performance_index[-1]
    average_performance_cond = (y_test > percentiles_performance_index[0]) & (y_test < percentiles_performance_index[-1])
    low_performance_cond = y_test <= percentiles_performance_index[0]

    high_performance = np.where(high_performance_cond)[0]
    average_performance = np.where(average_performance_cond)[0]
    low_performance = np.where(low_performance_cond)[0]
    
    high_performance_y =  y_test[high_performance]
    average_performance_y = y_test[average_performance]
    low_performance_y = y_test[low_performance]

    high_performance_x =  x_test[high_performance]
    average_performance_x = x_test[average_performance]
    low_performance_x = x_test[low_performance]
    
    high_performance_pred =  model.predict(x_test[high_performance])
    average_performance_pred = model.predict(x_test[average_performance])
    low_performance_pred = model.predict(x_test[low_performance])
    
    high_performance_pred = high_performance_pred*100
    average_performance_pred = average_performance_pred*100
    low_performance_pred = low_performance_pred*100
    
    mae_high_perf = mean_absolute_error(high_performance_y, high_performance_pred)
    mae_average_perf = mean_absolute_error(average_performance_y, average_performance_pred)
    mae_low_perf = mean_absolute_error(low_performance_y, low_performance_pred)
    return f"MAE high performance students:{mae_high_perf}\nMAE average performance students:{mae_average_perf}\nMAE low performance students:{mae_low_perf}"
    

In [ ]:
accuracy()
print(stratified_accuracy())